In [3]:
import numpy as np
from scipy.sparse import csr_matrix as csr
from scipy.sparse import eye

# LU Decomposition

In [4]:
def LU_decompse(A): # LU разложение Вход 2х мерный массив, Выход 2х мерные массивы
    n_rows = len(A)
    n_cols = len(A[0])
    L = np.eye(n_rows, n_cols)
    U = np.zeros((n_rows, n_cols))
    for i in range(n_rows):
        for j in range(n_cols):
            if i <= j:
                _s = 0
                for k in range(i):
                    _s += L[i][k] * U[k][j]
                U[i][j] = A[i][j] - _s
            else:
                _s = 0
                for k in range(j):
                    _s += L[i][k] * U[k][j]
                L[i][j] = (A[i][j] - _s) / U[j][j]
    return L, U

In [5]:
def LU_decompse_csr(A): # LU разложение Вход csr матрица Выход csr матрица (не оптимально)
    n_rows, n_cols = A.shape
    L = eye(n_rows, n_cols, format="csr")
    U = csr((n_rows, n_cols), dtype=float)
    for i in range(n_rows):
        for j in range(n_cols):
            if i <= j:
                _s = 0
                for k in range(i):
                    _s += L[i, k] * U[k, j]
                U[i, j] = A[i, j] - _s
            else:
                _s = 0
                for k in range(j):
                    _s += L[i, k] * U[k, j]
                L[i, j] = (A[i, j] - _s) / U[j, j]
    return L, U

In [6]:
def LU_decompse_csr_full(A):
    n_rows, n_cols = A.shape
    l_data = []
    l_indices = []
    l_indptr = []
    u_data = []
    u_indices = []
    u_indptr = []
    for i in range(n_rows):
        l_indptr.append(len(l_data))
        u_indptr.append(len(u_data))
        for j in range(n_cols):
            if i <= j:
                _s = 0
                if i == j:
                    l_data.append(1)
                    l_indices.append(j)
                for x, y in zip(l_data[l_indptr[i]:], list(map(lambda x: x[0], filter(lambda x: x[1] == j, zip(u_data, u_indices))))):
                    _s += x * y
                u_data.append(A[i, j] - _s)
                u_indices.append(j)
            else:
                _s = 0
                for x, y in zip(l_data[l_indptr[i]:], list(map(lambda x: x[0], filter(lambda x: x[1] == j, zip(u_data, u_indices))))):
                    _s += x * y
                l_data.append((A[i, j] - _s) / u_data[u_indptr[j]])
                l_indices.append(j)
    l_indptr.append(len(l_data))
    u_indptr.append(len(u_data))
    return csr((l_data, l_indices, l_indptr)), csr((u_data, u_indices, u_indptr))

In [8]:
A = np.array([[10, -7, 0], 
              [-3, 6, 2], 
              [5, -1, 5]])
sA = csr(A)
print("sA")
print(sA)

# print(LU_decompse(A))
# l, u =  LU_decompse_csr(sA)
# print("L: ", l)
# print("U: ", u)

# l, u = LU_decompse_csr_full(sA)
# print("L: ", l.toarray())
# print("U: ", u.toarray())

sA
  (0, 0)	10
  (0, 1)	-7
  (1, 0)	-3
  (1, 1)	6
  (1, 2)	2
  (2, 0)	5
  (2, 1)	-1
  (2, 2)	5


# Usage of sparse matrices

In [9]:
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
mtx = csr((data, (row, col)), shape=(3, 3))
mtx     

<3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [18]:
mtx.todense()

matrix([[1, 0, 2],
        [0, 0, 3],
        [4, 5, 6]])

In [19]:
mtx.data

array([1, 2, 3, 4, 5, 6])

In [20]:
mtx.indices

array([0, 2, 2, 0, 1, 2], dtype=int32)

In [21]:
mtx.indptr

array([0, 2, 3, 6], dtype=int32)

# Solving a system of linear equations

## Additional functions

In [10]:
# get i-th row of sparse matrix A

def get_row(A, i):
    _, n = A.shape
    row = np.zeros(n)
    non_zero_elements_in_row = A.data[A.indptr[i]:A.indptr[i + 1]]
    indices_of_non_zero_elements_in_row = A.indices[A.indptr[i]:A.indptr[i + 1]]
    k = 0
    for i in range(n):
        if k < len(non_zero_elements_in_row) and i == indices_of_non_zero_elements_in_row[k]:
            row[i] = non_zero_elements_in_row[k]
            k += 1
    return row

## Solve function

In [12]:
# A: sparse matrix
# b: array of values

def solve(A, b):
    L, U = LU_decompse_csr_full(A)
    _, n = A.shape
    
    # solving Ly = b
    y = np.zeros(n)
    for i in range(n):
        row = get_row(L, i)
        y[i] = b[i] - np.dot(y[:i], row[:i])
    
    # solving Ux = y
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        row = get_row(U, i)
        x[i] = (y[i] - np.dot(x[i + 1:], row[i + 1:])) / row[i]
        
    return x

## Example

In [13]:
A = np.array([[10, -7, 2], 
              [-3, 4, 6], 
              [5, -1, 5]])
A_sparse = csr(A)
b = [1, 2, 3]
x = solve(A_sparse, b)
print("Solution: ", x)

Solution:  [0.65168539 0.82022472 0.11235955]


# Finding inverse matrix

In [14]:
def inverse(A):
    _, n = A.shape
    I = np.eye(n)
    A_inverse = np.zeros(A.shape)
    for i in range(n):
        x = solve(A, I[:, i])
        A_inverse[:, i] = x
    return csr(A_inverse)

## Example

In [102]:
A = np.array([[10, -7, 2], 
              [-3, 4, 6], 
              [5, -1, 5]])
A_sparse = csr(A)
A_inverse = inverse(A)
print(A_inverse)

  (0, 0)	-0.2921348314606741
  (0, 1)	-0.3707865168539326
  (0, 2)	0.5617977528089887
  (1, 0)	-0.5056179775280898
  (1, 1)	-0.44943820224719105
  (1, 2)	0.7415730337078651
  (2, 0)	0.19101123595505617
  (2, 1)	0.2808988764044944
  (2, 2)	-0.21348314606741572


# Gilbert Matrices

In [25]:
def generate_gilbert_matrix(k):
    matrix = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            matrix[i][j] = 1 / (i + j + 1)
    return matrix


def solve_system(k):
    A = generate_gilbert_matrix(k)
    x = np.zeros((k))
    for i in range(k):
        x[i] = i + 1
    F = np.dot(A, x)
    x_new = solve(csr(A), F)
    error = np.linalg.norm(x - x_new)
    return x_new, error
    

def solve_systems(k):
    error_array = []
    x_new_array = []
    for i in range(1, k + 1):
        x_new, error = solve_system(i)
        x_new_array.append(x_new)
        error_array.append(error)
        print("k =", i, "\terror =", round(error, 5), "\tx' =", x_new, "\n")
    return error_array, x_new_array
    

solve_systems(13)

k = 1 	error = 0.0 	x' = [1.] 

k = 2 	error = 0.0 	x' = [1. 2.] 

k = 3 	error = 0.0 	x' = [1. 2. 3.] 

k = 4 	error = 0.0 	x' = [1. 2. 3. 4.] 

k = 5 	error = 0.0 	x' = [1. 2. 3. 4. 5.] 

k = 6 	error = 0.0 	x' = [1. 2. 3. 4. 5. 6.] 

k = 7 	error = 0.0 	x' = [1.         2.         2.99999999 4.00000003 4.99999994 6.00000006
 6.99999998] 

k = 8 	error = 0.0 	x' = [1.         2.         3.00000005 3.99999975 5.00000066 5.99999909
 7.00000064 7.99999982] 

k = 9 	error = 0.00016 	x' = [1.         1.99999989 3.0000018  3.99998727 5.00004631 5.99990609
 7.00010725 7.99993553 9.00001587] 

k = 10 	error = 0.00041 	x' = [ 1.          2.00000005  2.99999885  4.00001073  4.99994742  6.0001482
  6.99975132  8.00024521  8.99986893 10.00002929] 

k = 11 	error = 0.0345 	x' = [ 0.99999999  2.00000144  2.99996419  4.0003869   4.99776213  6.0076678
  6.98367917  8.0218068   8.98220856 10.00809994 10.99842307] 

k = 12 	error = 3.89577 	x' = [ 0.99999976  2.00003     2.99905972  4.01277547  4.9065